<a href="https://colab.research.google.com/github/saribasmetehan/Fine-tuning-Mistral-7B-using-LoRA-technique/blob/main/practice_of_using_the_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U peft bitsandbytes

In [4]:
from peft import PeftConfig

In [5]:
peft_model_name = "saribasmetehan/mistral-7b-turkish-finance"

In [6]:
config = PeftConfig.from_pretrained(peft_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

In [7]:
from transformers import BitsAndBytesConfig

In [8]:
from transformers import AutoModelForCausalLM

In [9]:
!pip install -q huggingface_hub

In [10]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
!pip install -q accelerate

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_4bit=True,
    device_map="auto"
)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [13]:
from transformers import AutoTokenizer

In [14]:
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path,
    padding_side="right",
    add_eos_token=True,
)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [15]:
from peft import PeftModel

In [16]:
fine_tune_model = PeftModel.from_pretrained(model, peft_model_name)

adapter_model.safetensors:   0%|          | 0.00/609M [00:00<?, ?B/s]

In [17]:
from transformers import pipeline, logging

In [18]:
import torch

In [19]:
logging.set_verbosity(logging.CRITICAL)

pipe = pipeline(
    "text-generation",
    model=fine_tune_model,
    tokenizer=tokenizer,
    eos_token_id=model.config.eos_token_id,
    max_new_tokens = 100
)

In [20]:
promt = """Enflasyon nedir?"""

In [21]:
result = pipe(f"<s>[INST]{promt}[/INST]")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [22]:
generated = result[0]["generated_text"]
print(generated[generated.find("[/INST]")+8:])


Enflasyon, bir ekonomideki yatırım ve tüketim seviyelerinin yükselmesi ve bir dönem içinde bir dönem içinde bir dönem içinde bir dönem içinde bir dönem içinde bir dönem içinde bir dönem içinde bir dönem iç


In [23]:
generated

'<s>[INST]Enflasyon nedir?[/INST]\n\nEnflasyon, bir ekonomideki yatırım ve tüketim seviyelerinin yükselmesi ve bir dönem içinde bir dönem içinde bir dönem içinde bir dönem içinde bir dönem içinde bir dönem içinde bir dönem içinde bir dönem iç'